# Chapter 2: Event Sampling & Filters
AFML focus: standard/event bars and CUSUM-style event extraction.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
idx = openquant.filters.cusum_filter_indices(uso, 0.001)
z_idx = openquant.filters.z_score_filter_indices(uso, 20, 20, 1.5)

plt.figure(figsize=(10,4))
plt.plot(uso, label='USO close', lw=1.2)
plt.scatter(idx, [uso[i] for i in idx], s=10, label='CUSUM events')
plt.title('Chapter 2: Event Sampling with CUSUM')
plt.legend()
plt.tight_layout()
plt.show()

print('cusum events', len(idx), 'z-score events', len(z_idx))


cusum events 845 z-score events 108


## Interpretation
CUSUM produces dense event sets on this sample, consistent with AFML event-driven workflows.
Z-score events are sparser and can be used as an alternative trigger policy.

